# Summary

Crawl a bunch of sites and save data to BQ

In [ ]:
import sys, os

import pandas as pd
import numpy as np

# Asynchronous work
import asyncio

from google.oauth2 import service_account
import pandas_gbq

# Web crawling and scraping tools class
sys.path.insert(0, "../../scrapers/")
import web_scraper as ws
import web_crawler as wc


## Test the site scraper on one site

In [ ]:
turl = "https://en.wikipedia.org/wiki/Chaffey_College"
turl = "https://asdjhfbads,jhbf"
turl = "https://www.cccco.edu/"

test = await ws.webScraper.visit_page(url=turl)

test.crawl_results.keys()
# type(test.crawl_results)


## Web Crawler

## Set up GCP credentials

In [ ]:
# Set GCP credentials
creds_path = "../../../data/environment/"
creds_file = "eternal-bongo-435614-b9-bf6a5e630e44.json"
credentials = service_account.Credentials.from_service_account_file(filename=os.path.join(creds_path,
                                                                                          creds_file))

# Set credentials
pandas_gbq.context.credentials = credentials


## Get a list of previously crawled websites

In [ ]:
# Query the BigQuery crawl results table to get a list of sites already crawled
# Identify the table; note the database.table naming schema
table_id = "ccc_polasst.crawl_data"
project_id = "eternal-bongo-435614-b9"

# Create a sql statement
sql = ("SELECT DISTINCT url FROM `{}` ").format(table_id)
sql = ("SELECT * FROM `{}` ").format(table_id)

# Use Pandas-gbq to read from the BQ table
# df_pc = pandas_gbq.read_gbq(sql, project_id=project_id)

########### Get a list of previously crawled sites
########### This needs some work - don't crawl may not work since we still need the URLs
# crawled_urls = df_pc["url"].unique().tolist()

# dont_crawl_urls = []

# len(crawled_urls)

## Crawl websites

In [ ]:
# Crawl seed_url and web links found on it and its child pages
seed_url = "https://www.cccco.edu"
# seed_url = "https://en.wikipedia.org/wiki/California_Community_Colleges"
dont_crawl_urls = []

crawler = wc.webCrawler(seed_url=seed_url)
df = await crawler.crawl_sites(dont_crawl_urls=dont_crawl_urls,
                               depth=1,
                               width=2)


In [ ]:
df
print("The crawl dataframe is {} rows.".format(len(df)))


## Read local crawl results

In [ ]:
crawl_data_path  = ("/Users/stephengodfrey/OneDrive - numanticsolutions.com"
                    "/Engagements/Projects/ccc_policy_assistant/data/crawls")
crawl_file = "wwwccccoedu_2024Dec11_1.csv"

os.listdir(crawl_data_path)


In [ ]:
# Read data file to test save
dft2 = pd.read_csv(filepath_or_buffer=os.path.join(crawl_data_path, crawl_file))
dft2

## Save results to BigQuery

In [ ]:
# Identify the table; note the database.table naming schema
table_id = "ccc_polasst.crawl_data"
table_id = "ccc_polasst.crawl_data_tes"
project_id = "eternal-bongo-435614-b9"

# Use Pandas-gbq to write to the BQ table
if_exists = "replace"
# if_exists = "append"

# pandas_gbq.to_gbq(dataframe=df, destination_table=table_id,
#                   project_id=project_id, if_exists=if_exists)

